# **Inferencia estadística**
#IE09. Ejercicios y desafíos adicionales




<img src="https://drive.google.com/uc?export=view&id=1Igtn9UXg6NGeRWsqh4hefQUjV0hmzlBv" width="100" align="left" title="Runa-perth">
<br clear="left">
Contenido opcional

<font color='red'>__Recomendación__</font>: Te animamos a intentar resolver los ejercicios y desafíos solo y, a continuación, buscar comparar resultados y perfeccionar los trabajos de forma grupal.

## <font color='blue'>**Caso Práctico - Desafío para los Inversores**</font>

### <font color='blue'>**El desafío**</font>

Un desafío común al que se enfrenta un inversor es determinar si es probable que una acción cumpla con sus objetivos de inversión. Por ejemplo, considere a Pepe, un inversionista que busca comprar acciones seguras durante los próximos cinco años.

Al ser un inversor reacio al riesgo, solo se sentiría cómodo invirtiendo en una acción si ésta le brinda retornos mensuales preferiblemente no negativos en promedio y si puede estar razonablemente seguro de que su retorno mensual promedio no será inferior al -0,5%.

Pepe está muy interesado en las acciones de ITC (NSE: ITC), pero quiere asegurarse de que cumpla con su criterio. En tal caso, ¿existe algún marco teórico / estadístico que pueda ayudarla a llegar a una conclusión sobre el rendimiento mensual promedio de las acciones de ITC con cierta confianza?

¡La respuesta es sí! Y dicho marco estadístico se basa en el supuesto de la distribución de rendimiento, que a su vez se basa en el Teorema del Límite Central (CLT).

### <font color='blue'>**El supuesto de normalidad en las finanzas**</font>

En los modelos financieros, generalmente asumimos que la serie de precios tiene una distribución logarítmica normal y, por lo tanto, los rendimientos se distribuyen normalmente. Y así, para los inversores, que utilizan estos modelos para invertir, mucho depende de si el supuesto de normalidad se cumple o no en los mercados.

Echemos un vistazo a las acciones de ITC (NSE: ITC) en las que Pepe está interesado. Para realizar el análisis, primero importamos algunas bibliotecas estándar de Python y obtenemos los datos de acciones de ITC de Close-Close diarios de la biblioteca de finanzas.

In [ ]:
!pip install yfinance
!pip install pyfolio

In [ ]:
# Librerias necesarias
import datetime as dt
import yfinance as yf
#import pyfolio as pf
import pandas as pd
import numpy as np
import seaborn as sns
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_rows', None)

# Obtención de los datos de acciones del ITC de la biblioteca yfinance de los últimos 10 años
start = dt.datetime(2010,1,1).date()
end = dt.datetime(2020,12,31).date()
ITC = yf.download('ITC.NS',start, end)[['Adj Close']]

daily_data = ITC.copy().round(4)

In [ ]:
# Echando un vistazo a los datos obtenidos
daily_data.head()

In [ ]:
np.log(62.9461 / 62.3318)

In [ ]:
daily_data['Adj Close'].head() / daily_data['Adj Close'].head().shift()

In [ ]:
np.log(daily_data['Adj Close'].head() / daily_data['Adj Close'].head().shift())

In [ ]:
# Cálculo de retornos diarios
daily_data['daily_return']= np.log(daily_data['Adj Close']/daily_data['Adj Close'].shift())
daily_data.dropna(inplace=True)

# Echando un vistazo a los datos obtenidos
daily_data.head()

Ahora que tenemos las retornos diarios de ITC (en escala logarítmica), en los siguientes diagramas visualizamos tanto los retornos como su distribución:

In [ ]:
import matplotlib.pyplot as plt

# Visualización de los retornos del registro diario
sns.set(style="white", palette="muted", color_codes=True)
plt.figure(figsize=(15,5))

plt.subplot(1,2,1)
# Trace un histograma simple con el tamaño del bins determinado automáticamente
sns.lineplot(x=daily_data.index, y=daily_data['daily_return'], color="r")
plt.ylabel('Valores reales de daily_returns')
plt.title('Los valores reales de los retornos del registro diario a lo largo del tiempo')

plt.subplot(1,2,2)

sns.histplot(daily_data['daily_return'], kde=False, color="r")
plt.ylabel('Frequencia')


plt.tight_layout()
plt.title('Distribución de retornos diarios')

plt.show();

Podemos observar que en los diez años que estamos usando, el rendimiento diario de ITC se centra en 0. Sin embargo, en pocas ocasiones ha superado la marca del cinco por ciento al alza e incluso una marca negativa del diez por ciento a la baja. Esto explica la mayor **curtosis**, lo cual no es propio de una distribución normal.

Hasta ahora, nuestras conclusiones se basan en el análisis visual (muy recomendado como paso inicial). Para validarlos, necesitamos realizar una prueba estadística adecuada para verificar la normalidad/no normalidad de los datos, que es lo que haremos en la siguiente sección.

### <font color='blue'>**La prueba de Shapiro-Wilk**</font>

Podemos realizar una prueba estadística, la prueba de Shapiro-Wilk, para verificar la normalidad de los datos. Es la prueba más poderosa cuando se prueba una distribución normal.

La hipótesis nula plantea la normalidad de los datos. Si el valor $p$ es mayor que 0.01, asumimos una distribución normal con 99% de confianza, de lo contrario rechazamos la hipótesis nula.
<br>
$$H_0 : \text{La distribución de los datos es Normal}$$
$$H_a : \text{La distribución de los datos NO es Normal}$$
<br>

Ejecutamos la prueba mencionada anteriormente a continuación en los retornos de registro diarios usando la función `shapiro` de la biblioteca `scipy`:

In [ ]:
from scipy.stats import shapiro
stat1, p1 = shapiro(daily_data['daily_return'])

if p1 < 0.01:
    test_result1 = 'No es Normal'
    print(f'El p-value es {p1:5.3e} < 0.01, por lo tanto se rechaza que los datos se distribuyen normalmente con un 99% de confianza.')
    print(test_result1)
else:
    test_result1 = 'Es Normal'
    print(f'El p-value es {p1:5.3e} < 0.01, por lo tanto asumimos que los datos estan distribuidos normalmente con un 99% de confianza. ')
    print(test_result1)

### <font color='blue'>**El papel del Teorema del Límite Central**</font>

Recuerde que __TLC__ supone que X es una variable aleatoria (no necesariamente normal) que representa los datos de la población. Y que la distribución de
X, tiene una media de μ  y desviación estándar σ. Supongamos que estamos tomando muestras repetidas de tamaño _n_ de la población anterior. Entonces, para un tamaño de muestra _n_ suficientemente alto, las siguientes propiedades son verdaderas:

1. $$ \bar{x}\rightarrow N\left(\mu,\frac{\sigma}{\sqrt{n}}\right)$$

2. Pero

$$\bar{x} = \frac{x_1+x_2+..+x_n}{n}$$


3. Entonces

$$\frac{x_1+x_2+..+x_n}{n} \rightarrow N\left(\mu,\frac{\sigma}{\sqrt{n}}\right)$$

4. Por lo tanto

$$ [x_1+x_2+..+x_n] \rightarrow N(n\mu,\;\sigma\sqrt{n})$$

<br>
Por lo tanto, hemos demostrado que de acuerdo con el teorema del límite central, si $[x_1,x_2+..,x_n]$   es una muestra aleatoria de tamaño $n$ de una población de cualquier distribución, entonces la suma $[x_1+x_2+..+x_n]$ es tambien distribuida normalmente, dado un $n$ grande.

En nuestro caso, las muestras aleatorias repetidas no son más que rendimientos logarítmicos diarios consecutivos de la acción. Como los retornos de registros son de naturaleza aditiva, si tomamos muestras de los retornos de registros diarios para cada día de la semana (asumiendo que hay cinco días de negociación en la semana) y los sumamos, obtenemos el rendimiento semanal.

Del mismo modo, si sumamos los rendimientos del registro diario de todos los días de negociación en un mes, obtenemos el rendimiento mensual.




### <font color='blue'>**Prueba de la normalidad de las devoluciones semanales y mensuales**</font>

Uno de los aspectos principales al construir modelos financieros cuantitativos basados en los rendimientos de las acciones es que existe un proceso de generación de datos de población,  y estamos tomando muestras de esos datos.

Por ejemplo, asumimos que existe un proceso de generación de datos que genera retornos de acciones. Se desconocen los parámetros que definen la distribución de este proceso, pero es necesario estimarlos.

Si usamos retornos de registros diarios, estamos muestreando los datos al final de cada día de negociación del precio de cierre anterior. Por lo tanto, $x_1$
es el rendimiento del registro al final del primer día de negociación,
$x_2$ es el retorno del registro al final del segundo día de negociación y así sucesivamente. Ahora bien, si tomamos una muestra de $n$ días, tenemos una muestra de tamaño $n$. Ya hemos visto que las devoluciones de los registros diarios no se distribuyen normalmente.

Veamos qué sucede cuando analizamos los rendimientos semanales en lugar de los rendimientos diarios (es decir, aumentamos $n$ de 1 a 5).


In [ ]:
# Remuestreo para obtener datos de cierre-cierre semanal y retorno semanal
weekly_data = ITC.resample('W').last()
weekly_data['weekly_return'] = np.log(weekly_data['Adj Close'] / weekly_data['Adj Close'].shift())
weekly_data.dropna(inplace=True)
weekly_data.head()

In [ ]:
# Dibujando el valor actual y la distribución de retornos semanales.
sns.set(style="white", palette="muted", color_codes=True)
plt.figure(figsize=(14,6))

plt.subplot(1,2,1)
sns.lineplot(x=weekly_data.index, y=weekly_data['weekly_return'], color="g")
plt.ylabel('Valores reales de los retornos semanales')
plt.title('Valores reales de los retornos semanales(log)\n a lo largo del tiempo')

plt.subplot(1,2,2)
sns.histplot(weekly_data['weekly_return'], kde=False, color="g")
plt.ylabel('Frecuencia')
plt.title('Districión de los retornos semanales')

plt.tight_layout()
plt.show();

In [ ]:
from scipy.stats import shapiro
stat2, p2 = shapiro(weekly_data['weekly_return'])

if p2 < 0.01:
    test_result1 = 'No es Normal'
    print(f'El p-value es {p2:5.3e} < 0.01, por lo tanto se rechaza que los datos se distribuyen normalmente con un 99% de confianza.')
    print(test_result1)
else:
    test_result1 = 'Es Normal'
    print(f'El p-value es {p2:5.3e} < 0.01, por lo tanto asumimos que los datos estan distribuidos normalmente con un 99% de confianza. ')
    print(test_result1)

Un valor p pequeño indica que, incluso cuando tomamos muestras a nivel semanal, los datos no son normales.

### <font color='green'>Actividad 1</font>
Verifique si los rendimientos mensuales (es decir, aumentando el tamaño de la muestra a 20) se distribuyen normalmente.

In [ ]:
# Tu código aquí ...


<font color='green'>Fin actividad 1</font>

### <font color='blue'>**Intervalo de confianza**</font>

El __intervalo de confianza__ describe la variabilidad entre la medida obtenida en un estudio y la medida real de la población (el valor real). Corresponde a un rango de valores, cuya distribución es normal y en el cual se encuentra, con alta probabilidad, el valor real de una determinada variable. Esta _"alta probabilidad"_ se ha establecido por consenso en 95%. Así, un intervalo de confianza de 95% nos indica que dentro del rango dado se encuentra el valor real de un parámetro con 95% de certeza.

Volviendo al problema de inversión de Pepe, como podemos asumir con seguridad que los __rendimientos mensuales se distribuyen normalmente__, podemos utilizar el concepto estadístico de intervalos de confianza para obtener un rango de rendimiento mensual promedio con cierta confianza.

En general, bajo el supuesto de normalidad, el intervalo de confianza viene dado por:
<br><br>
$$ \bar{x} \pm z *\frac{s}{\sqrt{n}}$$
<br>
Donde $z$ es el _z-score_ asociado con un nivel determinado de confianza.

Entonces, si queremos obtener el intervalo de confianza del 95% para el rendimiento mensual promedio, ya que casi el 95% de los datos para una variable normal estándar se encuentra entre aproximadamente +/- 2 desviaciones estándar, el puntaje $z$ sería 1,96.

Además, como no conocemos la desviación estándar de la población $\sigma$, usaremos la desviación estándar de la muestra $s$ en su lugar.
En el siguiente código, calculamos el intervalo de confianza para el rendimiento mensual promedio:

In [ ]:
from scipy import stats

In [ ]:
z = stats.norm.ppf(0.05/2, loc=0, scale=1)
z

In [ ]:
mean = np.mean(monthly_data['monthly_return'])

std = np.std(monthly_data['monthly_return'],ddof=1)/ np.sqrt(len(monthly_data))

#95% Intervalo de confianza.
upper = mean + z * std
lower = mean - z * std


CI_95 = (100 * np.round(lower, 3), 100 * np.round(upper, 3))
print(f'Estimación del rendimiento mensual medio: {100 * mean:4.3f}%') # Lo expresamos como por ciento
print(f'Intervalo de confianza del 95% para el rendimiento mensual medio en %: {CI_95}')


In [ ]:
mu, sigma = stats.norm.fit(monthly_data['monthly_return'])

In [ ]:
norm_m = stats.norm(mu, sigma)
x = np.linspace(norm_m.ppf(0.0001),
                norm_m.ppf(0.9999), 100)
fp = norm_m.pdf(x) # Función de Probabilidad
plt.plot(x, fp)
plt.vlines(lower, 0, max(fp), color='r')
plt.vlines(upper, 0, max(fp), color='r')
plt.tight_layout()
plt.title("Distribución Normal de datos mensualizados")
plt.ylabel("Densidad de probabilidad")
plt.xlabel("valores")

plt.show()

<img src="https://drive.google.com/uc?export=view&id=1Igtn9UXg6NGeRWsqh4hefQUjV0hmzlBv" width="50" align="left" title="Runa-perth">
<br clear="left">